In [2]:
import keras
import numpy as np

In [3]:
from keras.layers import Input,Conv2D, Dense, Flatten, MaxPooling2D

In [4]:
from keras import Sequential, Model

In [5]:
from keras.layers import concatenate

In [6]:
#a basic inception block
#in this 3x3 and 5x5 convolutions don't have a 1x1 precurring layer
#this function is built just for demonstration, due to limited computational ability this layer is not added
def basic_inception_block(input_layer, f1,f3,f5):
    conv1=Conv2D(
        filters=f1,
        kernel_size=(1,1),
        padding='same',
        activation='relu'
    )(input_layer)
    
    
    
    conv3=Conv2D(
        filters=f3,
        kernel_size=(3,3),
        padding='same',
        activation='relu'
    )(input_layer)
    
    
    
    
    conv5=Conv2D(
        filters=f5,
        kernel_size=(5,5),
        padding='same',
        activation='relu'
    )(input_layer)
    
    
    pooling=MaxPooling2D(
        (3,3),
        strides=(1,1),
        padding='same'
    )(input_layer)
    
    
    output_layer=concatenate([conv1,conv3,conv5,pooling], axis=3)
    
    return output_layer 
    
    

In [7]:
input_layer=Input(shape=(32,32,3))
iblock1=basic_inception_block(input_layer, 64, 128,32)


flat=Flatten()(iblock1)

dense1= Dense(60, activation='relu')(flat)
dense2= Dense(10, activation='softmax')(dense1)

model = Model(inputs=input_layer, outputs=dense2)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   256         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 128)  3584        ['input_1[0][0]']                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 32, 32, 32)   2432        ['input_1[0][0]']                
                                                                                              

In [9]:
#data pre processing
import glob
from PIL import Image
from matplotlib import pyplot as plt
from random import shuffle
import cv2

train_files=glob.glob("C:/Users/Rohan/GitHubstuff/test_git/CNN-implementation-of-reasearch-models/fingers/train/*")
shuffle(train_files)

X_train=np.zeros((len(train_files),32,32,3))
Y_train=np.zeros((len(train_files)))

for i,fl in enumerate(train_files):
    image = cv2.imread(fl)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    backtorgb = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)
    
    backtorgb = cv2.resize(backtorgb,(32,32))
    backtorgb= backtorgb/255
    
    
    X_train[i,:,:,:]=backtorgb
        
    Y_train[i]=fl[-6]
    

In [10]:
# inception model with dimension reduction; less parameters using 1x1 convolutions precurring the 3x3 and 5x5 convolution layers
def reduced_dimension_inception_block(input_layer, f1, f3_in, f3_out, f5_in, f5_out, pool_out):
    
    conv1=Conv2D(
        filters=f1,
        kernel_size=(1,1),
        padding='same',
        activation='relu'
    )(input_layer)
    
    
    
    conv3=Conv2D(
        filters=f3_in,
        kernel_size=(1,1),
        padding='same',
        activation='relu'
    )(input_layer)
    
    conv3=Conv2D(
        filters=f3_out,
        kernel_size=(3,3),
        padding='same',
        activation='relu'
    )(conv3)
    
    
    
    
    conv5=Conv2D(
        filters=f5_in,
        kernel_size=(1,1),
        padding='same',
        activation='relu'
    )(input_layer)
    
    conv5=Conv2D(
        filters=f5_out,
        kernel_size=(5,5),
        padding='same',
        activation='relu'
    )(conv5)
    
    
    
    
    pooling=MaxPooling2D(
        (3,3),
        strides=(1,1),
        padding='same',
    )(input_layer)
    
    pooling=Conv2D(
        filters=pool_out,
        kernel_size=(1,1),
        padding='same',
        activation='relu'
    )(pooling)
    
    
    
    output_layer=concatenate([conv1,conv3,conv5,pooling], axis=3) #axis 3 is the concatenation axis
    
    return output_layer
    
    
    
   

In [11]:
input_layer=Input(shape=(32,32,3))
iblock1= reduced_dimension_inception_block(input_layer, 64, 64, 128, 16, 32, 32)


flat=Flatten()(iblock1)

dense1= Dense(60, activation='relu')(flat)
dense2= Dense(10, activation='softmax')(dense1)

model = Model(inputs=input_layer, outputs=dense2)

In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 32, 32, 64)   256         ['input_2[0][0]']                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 32, 32, 16)   64          ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 3)   0           ['input_2[0][0]']                
                                                                                            

In [13]:
model.compile(
    optimizer= keras.optimizers.Adam(learning_rate=0.001),
    loss= 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
model.fit(
    X_train,
    Y_train,
    validation_split=0.2,
    epochs=3)

Epoch 1/3
450/450 [==============================] - 77s 169ms/step - loss: 0.2501 - accuracy: 0.9365 - val_loss: 0.0515 - val_accuracy: 0.9811
Epoch 2/3
450/450 [==============================] - 86s 190ms/step - loss: 0.0068 - accuracy: 0.9987 - val_loss: 0.0024 - val_accuracy: 0.9997
Epoch 3/3
450/450 [==============================] - 115s 255ms/step - loss: 0.0013 - accuracy: 0.9999 - val_loss: 0.0024 - val_accuracy: 1.0000


In [ ]:
#we can observe remarkable accuracy of 100% with this model.
#not evaluation and predictions

In [15]:
test_files=glob.glob("C:/Users/Rohan/GitHubstuff/test_git/CNN-implementation-of-reasearch-models/fingers/test/*")


X_test=np.zeros((len(test_files),32,32,3))
Y_test=np.zeros((len(test_files)))

for i,fl in enumerate(test_files):
    image = cv2.imread(fl)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    backtorgb = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)
    
    backtorgb = cv2.resize(backtorgb,(32,32))
    backtorgb= backtorgb/255
    
    
    X_test[i,:,:,:]=backtorgb
        
    Y_test[i]=fl[-6]
    

In [16]:
model.evaluate(X_test, Y_test)

113/113 [==============================] - 4s 36ms/step - loss: 0.0018 - accuracy: 0.9997


[0.0018048506462946534, 0.9997222423553467]